Import Libraries

In [ ]:
#General Libraries
import numpy as np
import pandas as pd
# Visualization
import matplotlib.pyplot as plt
import seaborn as sn
#Data Scaling/Encoding Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
#Data Split/Oversampling/Model Tuning
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, GridSearchCV
# Models
from sklearn import tree, svm, ensemble, linear_model
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
import tensorflow as tf
import keras_tuner as kt
# Model Evaluation
from sklearn.utils import resample
from sklearn.metrics import roc_curve, auc, classification_report, roc_auc_score
#Model/Pipeline Export
import joblib
#Nomogram
from simpleNomo import nomogram
#Dcurve
from dcurves import dca, plot_graphs
#SHAP
import shap
shap.initjs()
#Random Seed
np.random.seed(42)

# Import and reformat Data

In [ ]:
#Import Data
df = pd.read_excel('./Data/Raw/ORN_Data.xlsx')
#Subset to only include relevant rows
df= df[:329]
## Rename for interpretability
df = df.rename(columns={'ORN': 'ORN_type', 'ORN ': 'ORN'})
##Get relevant columns
df_sub = df[[
             'GENDER (0=female, 1=male)',
             'AGE',
             'REOPEN',
             'ADMISSION( hospitalization days)',
             'MEDEXPOSURE( treat plate exposure with medication only)',
             'TXEXPOSURE(treatments other than medication, 1=plate removal, 2=another flap)',
             'PRIOREX( prior opearion at same side, 0=no, 1=yes)',
             'EXPOSUREFU( time from op to plate exposure)',
             'OPTIME',
             'ASA, 0= ASA I or II, 1=ASA III',
             'BMI',
             'DM(0=no, 1=yes)', 
             'RECUR(0=primary, 1=recurrence)', 
             'SECONDPRIMARY(0=no, 1=yes)',  
             'SITE(1= mouth floor,  2=buccal, 3=retromolar, 4=gum, 5=tongue , 6=lip)', 
             'T(0=T1or2, 1=T3or4)', 
             'N(0= (-), 1=(+))',
             'OVERALLSTAGE(1= stage1, 2=stage2/3, 3=stage4)',
             'DEFECTTYPE(0= intraoral only, 1=composite defect)',
             'JEWER(Jewer\'s classification, 0=C, 1=L, 3=LC, 5=LCL)',
             'LENGTH(defect length)',
             'OSTEOTOMY(no. of osteostomy of the fibula bone)',
             'PLATE(0=mini plate 1=reconstruction plate 2=preformed plate)',
             'FLAP(0=OSC flap, 1=chimeric flap with muscle)',
             'HGB(post-op hemoglobin)',
             'ALB(post-op albumin)',
             'BT(intra-op blood transfusion, 0=no, 1=yes)',
             'ISCHEMICTIME',
             'PRERT( pre-op RT)', 
             'POSTRT(post-op RT)', 
             'PRECT(pre-op chemotherapy)',
             'POSTCT(post-op chemotherapy)', 
             'WOUNDINF(post-op wound infection)',
             'EXPOSURE( plate exposure)',
             'ORN'
             ]]

##Rename columns for readability
df_renamed = df_sub.rename(columns = {
    'REOPEN': 'REOPEN',
    'ADMISSION( hospitalization days)': "ADMISSION",
    'MEDEXPOSURE( treat plate exposure with medication only)': 'MEDEXPOSURE',
    'TXEXPOSURE(treatments other than medication, 1=plate removal, 2=another flap)': "TXEXPOSURE",
    'PRIOREX( prior opearion at same side, 0=no, 1=yes)': 'PRIOREX',
    'EXPOSUREFU( time from op to plate exposure)': 'EXPOSUREFU',
    'GENDER (0=female, 1=male)': 'GENDER',
    'ASA, 0= ASA I or II, 1=ASA III': 'ASA',
    'DM(0=no, 1=yes)': 'DM', 
    'RECUR(0=primary, 1=recurrence)': 'RECUR', 
    'SECONDPRIMARY(0=no, 1=yes)': 'SP',  
    'SITE(1= mouth floor,  2=buccal, 3=retromolar, 4=gum, 5=tongue , 6=lip)': 'SITE', 
    'T(0=T1or2, 1=T3or4)': 'T', 
    'N(0= (-), 1=(+))': 'N',
    'OVERALLSTAGE(1= stage1, 2=stage2/3, 3=stage4)': 'STAGE',
    'DEFECTTYPE(0= intraoral only, 1=composite defect)': 'DEFECT_TYPE',
    'JEWER(Jewer\'s classification, 0=C, 1=L, 3=LC, 5=LCL)': 'JEWER',
    'LENGTH(defect length)': 'LENGTH',
    'OSTEOTOMY(no. of osteostomy of the fibula bone)': 'OSTEOTOMY',
    'PLATE(0=mini plate 1=reconstruction plate 2=preformed plate)': 'PLATE',
    'FLAP(0=OSC flap, 1=chimeric flap with muscle)': 'FLAP',
    'HGB(post-op hemoglobin)': 'HGB',
    'ALB(post-op albumin)': 'ALB',
    'BT(intra-op blood transfusion, 0=no, 1=yes)': 'BT',
    'PRERT( pre-op RT)': 'PRERT', 
    'POSTRT(post-op RT)': 'POSTRT', 
    'PRECT(pre-op chemotherapy)':'PRECT',
    'POSTCT(post-op chemotherapy)': 'POSTCT', 
    'WOUNDINF(post-op wound infection)': 'WOUNDINF',
    'EXPOSURE( plate exposure)':'EXPOSURE',
})

#Change the single 2 entry for exposure to a 1 for simplicity
df_renamed.EXPOSURE = np.where(df_renamed.EXPOSURE >= 1, 1.0, df_renamed.EXPOSURE)

#Extract features with NA or 999 entries
for col in df_renamed.columns:
    if col == 'OPTIME':
        continue
    na_vals = df_renamed[col].isna().sum()
    nine_vals = (df_renamed[col] == 999).sum()
    if na_vals > 0:
        print(f'{col}: {na_vals}  NA vals')
        print('--------')
    elif(nine_vals > 0):
        print(f'{col}: {nine_vals} 999 vals')
        print('--------')

In [ ]:
df_renamed = df_renamed[(df_renamed.ALB < 999.0)] ##Remove ALB na
df_renamed = df_renamed[(df_renamed.BMI < 999.0)] ##Remove BMI na
df_renamed = df_renamed[(df_renamed.ISCHEMICTIME < 999.0)] ##Remove ISCHEMICTIME na
df_renamed = df_renamed[df_renamed.LENGTH > 0] ##Remove Length na
#Ensure data is fully cleaned
print(f'Total NA vals: {df_renamed.isna().sum().sum()}')
print(f'Total 999 vals: {(df_renamed == 999).sum().sum()}')

# Pipeline

Train-Val-Test Split

In [ ]:
#65-35 train-test split with stratified distribution of ORN to No-ORN
raw_X_train, raw_X_test, raw_y_train, raw_y_test = train_test_split(df_renamed.drop(['ORN'], axis = 1),
                                                    df_renamed['ORN'], 
                                                    test_size=0.35,
                                                    random_state = 42, 
                                                    stratify=df_renamed.ORN
                                                    )

#Reset index
raw_X_train.reset_index(drop = True, inplace=True)
raw_y_train.reset_index(drop = True, inplace=True)
raw_X_test.reset_index(drop = True, inplace=True)
raw_y_test.reset_index(drop = True, inplace=True)
#Export raw train/test splits
raw_X_train.to_excel('./Data/Raw/Raw_X_train.xlsx', index = False)
raw_X_test.to_excel('./Data/Raw/Raw_X_test.xlsx', index = False)
raw_y_train.to_excel('./Data/Raw/Raw_y_train.xlsx', index = False)
raw_y_test.to_excel('./Data/Raw/Raw_y_test.xlsx', index = False)

#Initialize feature lists
binary_cols = [] #Binary categorical
numerical_cols = [] #Numerical
nominal_cols = [] #Nominal Multi-categorical
ordinal_cols = ['STAGE'] #Ordinal categorical

##Classify feature types
for col in raw_X_train.columns:
    if col == 'STAGE': #Stage is the only ordinal variable
        continue
    num_entries = len(raw_X_train[col].value_counts())
    if num_entries == 2:
        binary_cols.append(col)
    elif num_entries > 10 or col == 'OSTEOTOMY': #Osteotomy only has 4 distinct entries but numerical
        numerical_cols.append(col)
    else:
       nominal_cols.append(col)
#Ensure total entries in lists add up
len(binary_cols) + len(numerical_cols) + len(nominal_cols) + len(ordinal_cols) == raw_X_train.shape[1]

Pipeline

In [ ]:
#ML Model Pipeline
numerical_transformer = Pipeline(steps=[ 
    ('scaler', MinMaxScaler())
])
ml_categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(drop = 'first', sparse_output=False)), #Drop first col in one-hot encoding
])
ml_preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols), #Scale numerical
        ('cat', ml_categorical_transformer, nominal_cols) #Encode categorical
    ],
    remainder='passthrough' #Pass binary and ordinal through
)
ml_preprocessor.set_output(transform="pandas") #Ensure output is df

#Nomogram Pipeline
nomo_categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(sparse_output=False)), #DONT drop first col in one-hot encoding
])
nomogram_preprocessor = ColumnTransformer( #No numerical scaling
    transformers=[
        ('cat', nomo_categorical_transformer, nominal_cols),
    ],
    remainder='passthrough' #Pass numerical, binary, and ordinal through
) 
nomogram_preprocessor.set_output(transform="pandas") #Ensure output is df

###Fit on train data and transform both training and testing
nomo_train_transformed = nomogram_preprocessor.fit_transform(raw_X_train)
nomo_test_transformed = nomogram_preprocessor.transform(raw_X_test)
ml_train_transformed = ml_preprocessor.fit_transform(raw_X_train)
ml_test_transformed = ml_preprocessor.transform(raw_X_test)

###Remove added prefixes in df for readability
#ML
ml_train_transformed.columns = ml_train_transformed.columns.str.removeprefix("num__")
ml_train_transformed.columns = ml_train_transformed.columns.str.removeprefix("cat__")
ml_train_transformed.columns = ml_train_transformed.columns.str.removeprefix("remainder__")

ml_test_transformed.columns = ml_test_transformed.columns.str.removeprefix("num__")
ml_test_transformed.columns = ml_test_transformed.columns.str.removeprefix("cat__")
ml_test_transformed.columns = ml_test_transformed.columns.str.removeprefix("remainder__")
#NOMO
nomo_train_transformed.columns = nomo_train_transformed.columns.str.removeprefix("remainder__")
nomo_train_transformed.columns = nomo_train_transformed.columns.str.removeprefix("cat__")
nomo_train_transformed.columns = nomo_train_transformed.columns.str.removeprefix("bin__")

nomo_test_transformed.columns = nomo_test_transformed.columns.str.removeprefix("remainder__")
nomo_test_transformed.columns = nomo_test_transformed.columns.str.removeprefix("cat__")
nomo_test_transformed.columns = nomo_test_transformed.columns.str.removeprefix("bin__")

#Export scaled/econded dfs to Excel
ml_train_transformed.to_excel('./Data/ML_Pipeline/ml_train_transformed.xlsx', index = False)
ml_test_transformed.to_excel('./Data/ML_Pipeline/ml_test_transformed.xlsx', index = False)

nomo_train_transformed.to_excel('./Data/Nomogram/nomo_train_transformed.xlsx', index = False)
nomo_test_transformed.to_excel('./Data/Nomogram/nomo_test_transformed.xlsx', index = False)
#Save pipeline for future use
joblib.dump(ml_preprocessor, './Data/ML_Pipeline/ml_preprocessing_pipeline.pkl')

# SMOTE

In [6]:
##Apply SMOTE
smote = SMOTE(sampling_strategy='minority', random_state=42)
full_X_sm, full_y_sm = smote.fit_resample(ml_train_transformed, raw_y_train)
#Reset Index
full_X_sm.reset_index(drop = True, inplace=True)
full_y_sm.reset_index(drop = True, inplace=True)
#Export
full_X_sm.to_excel('./Data/ML_Smote/full_x_sm.xlsx', index = False)
full_y_sm.to_excel('./Data/ML_Smote/full_y_sm.xlsx', index = False)

# Modeling

SVM, RF, DT, XGB, KNN, Vote

In [ ]:
##NOTE: Initial param search space was more broad and spaced at beginning of tuning, an iterative cycle of GridSearchCV 
####### and manual tuning was used to arrive at final model params

###Param Search Grid
model_params = {
    'SVM': {
        'model': svm.SVC(shrinking = True, degree = 7, gamma = 'scale', kernel = 'poly', C=1, class_weight={0:1, 1:3.5}, probability=True), 
        'params': {
            #'class_weight': ['balanced',{0:1, 1:3.15},{0:1, 1:3.25}, {0:1, 1:3.5},{0:1, 1:3.75},{0:1, 1:3.85}],\
            #'C': [0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
            #'degree': [6,7,8,9,12],
            #'kernel':  ['linear', 'poly', 'rbf', 'sigmoid'],
            #'gamma': ['scale', 'auto'],
            #'shrinking': [True, False],            
        }
    },
    'RF': {
        'model': ensemble.RandomForestClassifier(criterion = 'log_loss', max_features='sqrt', max_depth = 7, min_samples_split=12,
                                                 class_weight = {0:1, 1:3}, n_estimators=22, min_samples_leaf=3),
        'params': {
            #'class_weight': ['balanced',{0:1, 1:2.75}, {0:1, 1:3}, {0:1, 1:4}],
            #'n_estimators': [16,18,20,22]
            #'criterion':  ["entropy", "log_loss"],
            #'max_depth': [5,6,7,8,9,10], ##############RETRAIN to make between 5-15#############
            #'min_samples_split': [8,9,10,11,12,13],
            #'min_samples_leaf': [3,5,6,7],
            #'max_features': ['sqrt', 'log2']
        }
    },
    'DT': {
        'model': tree.DecisionTreeClassifier(max_depth = 23, max_features='log2', min_samples_split=3, 
                                             class_weight = {0:1, 1:4.5}, criterion='entropy', min_samples_leaf=4),
        'params': {
            #'class_weight': ['balanced', {0:1, 1:4.25}, {0:1, 1:4.5}, {0:1, 1:4.75}],
            #'criterion':  ["gini", "entropy", "log_loss"],
            #'max_depth': [None, 22, 23,24],
            #'min_samples_split': [2,3,4,5,6],
            #'min_samples_leaf': [4,5,6,7,8],
            #'max_features': ['sqrt', 'log2', None]
            }
    },
    'XGB': {
        'model': xgb.XGBClassifier(objective = 'binary:logistic', eval_metric = 'auc', seed = 42, booster = 'dart',
                                    grow_policy = 'depthwise', tree_method = 'approx', sampling_method = 'uniform', 
                                    learning_rate = 0.1, max_depth = 3, scale_pos_weight = 2, min_split_loss =8, 
                                    min_child_weight = 5, max_delta_step = 0, reg_lambda = 0, reg_alpha = 0),
        'params': {
            #'booster': ['dart', 'gbtree', 'gblinear'],
            #'grow_policy': ['depthwise', 'lossguide'],
            #'tree_method': ['auto','approx'],
            #'scale_pos_weight': [2,ratio,4,5],
            #'learning_rate': [0, 0.1,0.3, 0.5],
            #'subsample': [0.5, 0.7, 1], 
            #'max_leaves': [0,15,20,25], 
            #'max_bin': [256, 300, 325],
            ##Prevent overfitting
            #'max_depth': [3,5,6,7],
            #'min_child_weight': [1,3,5],
            #'min_split_loss': [1,3,5,7,8], 
            #'max_delta_step': [0, 0.7,1,2,3,4,5],
            #'refresh_leaf': [0,1],
            #'reg_lambda': [0,2,4,5,6],
            #'reg_alpha': [0, 0.5,1,2],
            #'sampling_method': ['uniform', 'weighted'], 
            #'normalize_type': ['tree', 'forest'], 
            }
    },
    'KNN':{
        'model':  KNeighborsClassifier(algorithm = 'auto', weights = 'distance', metric = 'cosine', p =1, n_neighbors=40),
        'params':{
            #'n_neighbors': [40,42,43],
            #'weights': ['uniform', 'distance'],
            #'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
            #'metric': ['cityblock', 'cosine', 'euclidean', 'l1', 'l2', 'manhattan', 'nan_euclidean'],
            #'leaf_size': [25,30,35], #Only used for BallTree or KDTree algorithm
            #'p': [1,2,3],
        }
    }, 
    'VOTE': {
        'model': ensemble.VotingClassifier(voting = 'soft', estimators = [
            ('SVM', svm.SVC(shrinking = True, degree = 7, gamma = 'scale', kernel = 'poly', C=1, class_weight={0:1, 1:3.5}, probability=True)),
            ('random_forest', ensemble.RandomForestClassifier(criterion = 'log_loss', max_features='sqrt', max_depth = 7, min_samples_split=12,
                                                 class_weight = {0:1, 1:3}, n_estimators=22, min_samples_leaf=3)),
            ('XGBoost',xgb.XGBClassifier(objective = 'binary:logistic', eval_metric = 'auc', seed = 42, booster = 'dart',
                                    grow_policy = 'depthwise', tree_method = 'approx', sampling_method = 'uniform', 
                                    learning_rate = 0.1, max_depth = 3, scale_pos_weight = 2, min_split_loss =8, 
                                    min_child_weight = 5, max_delta_step = 0, reg_lambda = 0, reg_alpha = 0)),
            ('Tree', tree.DecisionTreeClassifier(max_depth = 23, max_features='log2', min_samples_split=3, 
                                             class_weight = {0:1, 1:4.5}, criterion='entropy', min_samples_leaf=4)),
            ('KNN',  KNeighborsClassifier(algorithm = 'auto', n_neighbors=40, weights = 'distance', metric = 'cosine', p =1))
        ], weights = [5,20,10,0.1,9]),
        'params': {
            #'voting': ['hard', 'soft'],
            #'weights': [[5,15,10,0.1,10], [5,20,10,0.1,9]]
        }
    }
}


def get_best_params(map_name): #Use CV to get best params on smote dataset
    param_map = model_params[map_name]
    clf = GridSearchCV(param_map['model'], param_map['params'], scoring = 'roc_auc', cv = 5, return_train_score=True)
    clf.fit(full_X_sm, full_y_sm)
    print(f'{map_name} Best 5-fold CV Score: {np.round(clf.best_score_, 4)}')
    return clf

svc_clf = get_best_params('SVM')
rf_clf = get_best_params('RF')
tree_clf = get_best_params('DT')
boost_clf = get_best_params('XGB')
knn_clf = get_best_params("KNN")
vote_clf = get_best_params('VOTE')

Neural Network

In [7]:
##Create sub-train and sub-val datasets from train set
X_temp, nn_X_val, y_temp, nn_y_val = train_test_split(ml_train_transformed, raw_y_train, test_size = 0.2, stratify = raw_y_train, random_state = 42)
#Oversample sub-train set
smote = SMOTE(sampling_strategy='minority', random_state=42)
nn_X_sm, nn_y_sm = smote.fit_resample(X_temp, y_temp)

#Export to excel
nn_X_sm.to_excel('./Data/NN/nn_X_sm.xlsx', index = False)
nn_y_sm.to_excel('./Data/NN/nn_y_sm.xlsx', index = False)
nn_X_val.to_excel('./Data/NN/nn_X_val.xlsx', index = False)
nn_y_val.to_excel('./Data/NN/nn_y_val.xlsx', index = False)

In [ ]:
#NOTE: Commented values next to hp vals denote selected values
def model_builder(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape = nn_X_sm.loc[0].shape))

    hp_activation = hp.Choice('activation', values = ['relu', 'tanh']) #relu
    hp_layer_1 = hp.Int('layer_1', min_value =201, max_value = 401, step = 50) #301
    hp_layer_2 = hp.Int('layer_2', min_value =1701, max_value = 1901, step = 50) #1801
    hp_layer_3 = hp.Int('layer_3', min_value =1301, max_value = 1501, step = 50) #1401

    hp_drop_1 = hp.Choice('drop_1', values = [0.0, 0.2, 0.4, 0.6, 0.8, 0.99999]) #0.8
    hp_drop_2 = hp.Choice('drop_2', values = [0.0, 0.2, 0.4, 0.6, 0.8, 0.99999]) # 0.0
    hp_drop_3 = hp.Choice('drop_3', values = [0.0, 0.2, 0.4, 0.6, 0.8, 0.99999]) #0.4

    hp_lr = hp.Choice('learning_rate', values = [0.005, 0.01, 0.015]) #0.01

    model.add(tf.keras.layers.Dense(units = hp_layer_1, activation=hp_activation))
    model.add(tf.keras.layers.Dropout(hp_drop_1)) 
    model.add(tf.keras.layers.Dense(units = hp_layer_2, activation=hp_activation))
    model.add(tf.keras.layers.Dropout(hp_drop_2))
    model.add(tf.keras.layers.Dense(units = hp_layer_3, activation=hp_activation))
    model.add(tf.keras.layers.Dropout(hp_drop_3)) 

    model.add(tf.keras.layers.Dense(units = 1, activation='sigmoid'))

    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=hp_lr),
                  loss = 'binary_crossentropy',
                  metrics = [
        tf.keras.metrics.AUC(name='auc_roc'),
    ])
    return model


tuner = kt.Hyperband(model_builder,
                     objective = kt.Objective('val_auc_roc', 'max'),
                     max_epochs = 150,
                     factor = 3,
                     directory = './nn_tune_dir',
                     project_name = 'x')

stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_auc_roc', patience = 5)

##Tune parameters
tuner.search(nn_X_sm, nn_y_sm, 
             epochs = 150, validation_data = (nn_X_val, nn_y_val),
             class_weight={0:1, 1:1.2},
             callbacks = [stop_early])
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

In [ ]:
##Train model with tuned hyper params
model_NN = tuner.hypermodel.build(best_hps)
history = model_NN.fit(nn_X_sm, nn_y_sm, 
             epochs = 150,
             validation_data = (nn_X_val, nn_y_val),
             class_weight={0:1, 1:1.2},
             )

In [ ]:
###Plot AUROC and loss over epochs
# AUROC
val_auc_roc = history.history['val_auc_roc']
train_auc_roc = history.history['auc_roc']
plt.figure(figsize=(10, 6))
plt.plot(val_auc_roc, label='Validation AUC-ROC')
plt.plot(train_auc_roc, label='Train AUC-ROC')
plt.xlabel('Epochs')
plt.ylabel('AUC-ROC')
plt.title('AUC-ROC Over Epochs')
plt.legend()
plt.grid(True)
plt.show()

#Loss
val_loss = history.history['val_loss']
train_loss = history.history['loss']
plt.figure(figsize=(10, 6))
plt.plot(val_loss, label='Validation Loss')
plt.plot(train_loss, label='Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Loss Over Epochs')
plt.legend()
plt.grid(True)
plt.show()


# Nomogram

In [ ]:
##Train log model
log_model = linear_model.LogisticRegression(max_iter = 6000)
log_model.fit(nomo_train_transformed, raw_y_train)

##Get coef and intercept from log model
coefs = log_model.coef_[0]
intercept = log_model.intercept_[0]
##Create features and coef columns
feature_names = nomo_train_transformed.columns.to_list()
features = ['intercept'] + ['threshold'] + feature_names
coefs = [intercept] + [0.5] + list(coefs)

##Get continuous, nominal, and ordinal columns
cat_cols = [0] * 46 ## 0 = numerical
for i in range(0,46):
    col = nomo_train_transformed.columns[i]
    counts = nomo_train_transformed[col].value_counts()
    if col in nominal_cols or col in binary_cols: #1 = #Nominal
        cat_cols[i] = 1
    elif col in ordinal_cols: ##2 = Ordinal
        cat_cols[i] = 2

##Get mins, maxs, coefs, types, positions columns
mins = [''] + [''] +[nomo_train_transformed[feature].min() for feature in feature_names]
maxs = [''] + [''] +[nomo_train_transformed[feature].max() for feature in feature_names]
coefs = [1e-10 if i == 0 else i for i in coefs] #Make 0 entries a very small number
types = [''] +  [''] + ['numerical' if i == 0 else 'nominal' if i ==1 else 'ordinal' for i in cat_cols]
positions = [''] * 48

##Create table
nomo_table = pd.DataFrame({
    'feature': features,
    'coef': coefs,
    'min': mins,
    'max': maxs,
    'type': types,
    'position': positions
})

##Export and import table
path_to_nomo_table = './Data/Nomogram/Nomo_Table.xlsx'
nomo_table.to_excel(path_to_nomo_table, index = False)

##Create nomogram
results = nomogram(path = path_to_nomo_table, result_title = 'ORN Risk', 
                   fig_width=30, single_height=.7, dpi = 500,
                   ax_para={'linewidth': 2, 'c': 'black', 'linestyle': '--'},
                   xtick_para = {'fontsize': 13}
                   )

# Evaluate

In [13]:
def get_cm(model_name, y_test, y_pred):
    cm = tf.math.confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10,7))
    sn.heatmap(cm, annot=True, fmt ='d')
    plt.xlabel('Predicted')
    plt.ylabel('Truth')
    plt.title(model_name + ' Confusion Matrix')
    plt.show()

def bootstrap_auc(y_true, y_pred, n_bootstraps=3000, seed=42):
    np.random.seed(seed)
    bootstrapped_scores = []
    for i in range(n_bootstraps):
        # Sample predictions (w/ replcement)
        indices = resample(np.arange(len(y_pred)), replace=True)
        # ensures resampled data contains both classes
        if len(np.unique(y_true[indices])) < 2:
            continue
        # Calculate AUC for resampled data
        score = roc_auc_score(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    confidence_lower = sorted_scores[int(
        0.025 * len(sorted_scores))]  # 2.5th percentile
    confidence_upper = sorted_scores[int(
        0.975 * len(sorted_scores))]  # 97.5th percentile
    return confidence_lower, confidence_upper

def auc_CI(y_true, y_pred, n_bootsraps=3000, seed=42):
    score = roc_auc_score(y_true, y_pred)
    CI_lower, CI_upper = bootstrap_auc(y_true, y_pred, n_bootsraps, seed)
    return score, CI_lower, CI_upper

def evaluate(model, model_name, X_train, y_train, X_test, y_test, include_training = True, threshold = 0.5):   
    ########## Get model prob pred and pred based on threshold ########## 
    model_scores = {}
    if (model_name == 'Neural Network'):  # NN doesn't have predict_proba() method
        y_pred_proba_test = model.predict(X_test).ravel()
        y_pred_proba_train = model.predict(X_train).ravel()
    else:
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]
        y_pred_proba_train = model.predict_proba(X_train)[:,1]
    y_pred = (y_pred_proba_test >= threshold).astype('int')

    ##################### Get classification report #####################
    #Only for testing
    class_report = classification_report(y_test, y_pred)
    print(class_report)
    ##################### Get Confusion Matrix #########################
    #Only for testing
    get_cm(model_name, y_test, y_pred)
    ##################### ROC CURVE and AUROC #########################
    ##Training
    plt.figure(figsize=(12, 8))
    if include_training == True:
        roc_auc, lower_CI, upper_CI = auc_CI(y_train, y_pred_proba_train)
        model_score = f'AUROC = {roc_auc:.3f} ({lower_CI:.3f}-{upper_CI:.3f})'
        model_scores[model_name] = model_score
        fpr, tpr, thresholds = roc_curve(y_train, y_pred_proba_train)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=4, label=f'Training {model_score}')
    ##Testing
    roc_auc, lower_CI, upper_CI = auc_CI(y_test, y_pred_proba_test)
    model_score = f'AUROC = {roc_auc:.3f} ({lower_CI:.3f}-{upper_CI:.3f})'
    model_scores[model_name] = model_score
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba_test)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=4, label=f'Testing {model_score}')
    ####### Plot graphs ######
    plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate', fontsize = 21, fontweight = 550)
    plt.ylabel('True Positive Rate', fontsize = 21, fontweight = 550)
    plt.tick_params(axis = 'both', which = 'major', labelsize=15)
    plt.title(f'{model_name}', fontweight='semibold', fontsize = 25)
    plt.legend(loc="lower right", prop = {'size': 21, 'weight': 550})
    plt.show() 


Get CM, ROC curve, AUROC, CIs for AUROC

In [ ]:
evaluate(svc_clf, 'SVM', full_X_sm, full_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(rf_clf, 'Random Forest', full_X_sm, full_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(tree_clf, 'Decision Tree', full_X_sm, full_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(boost_clf, 'XGBoost', full_X_sm, full_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(knn_clf, 'KNN', full_X_sm, full_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(vote_clf, 'Vote', full_X_sm, full_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(model_NN, 'Neural Network', nn_X_sm, nn_y_sm, ml_test_transformed, raw_y_test)

In [ ]:
evaluate(log_model, 'Nomogram', nomo_train_transformed, raw_y_train, nomo_test_transformed, raw_y_test)

# DCurve

In [22]:
def get_dcurve(model, model_name, X_test, y_test):
    #Get predictions
    if (model_name == 'Neural Network'):  # Different pred method for NN
        y_pred_proba_test = model.predict(X_test).ravel()
    else:
        y_pred_proba_test = model.predict_proba(X_test)[:, 1]

    #Create new df with added predicted prob of ORN and ORN occurance cols
    df = X_test.copy()
    df['Probability'] = y_pred_proba_test
    df['ORN'] = y_test
    
    results = dca(
        data=df,
        outcome='ORN',
        modelnames=['Probability'],
        thresholds=np.arange(0, 1, 0.05)
    )
    
    plot_graphs(
        plot_df = results,
        y_limits = [-0.05, 0.4],
        graph_type='net_benefit'
    )
    
    return results

Get decision curves using testing data

In [ ]:
results = get_dcurve(rf_clf, 'Random Forest', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(tree_clf, 'Decision Tree', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(boost_clf, 'XGBoost', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(knn_clf, 'KNN', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(vote_clf, 'Vote', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(model_NN, 'Neural Network', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(model_NN, 'Neural Network', ml_test_transformed, raw_y_test)

In [ ]:
results = get_dcurve(log_model, 'Nomogram', nomo_test_transformed, raw_y_test)

# SHAP

In [31]:
# NOTE: combine_encoded() function taken from following repository:
# https://gist.github.com/peterdhansen/ca87cc1bfbc4c092f0872a3bfe3204b2#

######## Combine one-hot-encoded #######
def combine_encoded(shap_values, name, mask, return_original=True):
    mask = np.array(mask)
    mask_col_names = np.array(shap_values.feature_names, dtype='object')[mask]
    sv_name = shap.Explanation(shap_values.values[:, mask],
                               feature_names=list(mask_col_names),
                               data=shap_values.data[:, mask],
                               base_values=shap_values.base_values,
                               display_data=shap_values.display_data,
                               instance_names=shap_values.instance_names,
                               output_names=shap_values.output_names,
                               output_indexes=shap_values.output_indexes,
                               lower_bounds=shap_values.lower_bounds,
                               upper_bounds=shap_values.upper_bounds,
                               main_effects=shap_values.main_effects,
                               hierarchical_values=shap_values.hierarchical_values,
                               clustering=shap_values.clustering,
                               )
    new_data = (sv_name.data * np.arange(sum(mask))).sum(axis=1).astype(int)
    svdata = np.concatenate([
        shap_values.data[:, ~mask],
        new_data.reshape(-1, 1)
    ], axis=1)

    if shap_values.display_data is None:
        svdd = shap_values.data[:, ~mask]
    else:
        svdd = shap_values.display_data[:, ~mask]

    svdisplay_data = np.concatenate([
        svdd,
        mask_col_names[new_data].reshape(-1, 1)
    ], axis=1)

    new_values = sv_name.values.sum(axis=1)
    svvalues = np.concatenate([
        shap_values.values[:, ~mask],
        new_values.reshape(-1, 1)
    ], axis=1)
    svfeature_names = list(np.array(shap_values.feature_names)[~mask]) + [name]

    sv = shap.Explanation(svvalues,
                          base_values=shap_values.base_values,
                          data=svdata,
                          display_data=svdisplay_data,
                          instance_names=shap_values.instance_names,
                          feature_names=svfeature_names,
                          output_names=shap_values.output_names,
                          output_indexes=shap_values.output_indexes,
                          lower_bounds=shap_values.lower_bounds,
                          upper_bounds=shap_values.upper_bounds,
                          main_effects=shap_values.main_effects,
                          hierarchical_values=shap_values.hierarchical_values,
                          clustering=shap_values.clustering,
                          )
    if return_original:
        return sv, sv_name
    else:
        return sv
    

def get_shap(model, model_name, X_train, show_initial = False, show_sub_plots = False, display_df = False):
    if model_name == 'Neural Network': # Different method for neural network
        explainer = shap.Explainer(model, X_train)
    else:
        explainer = shap.Explainer(model.predict, X_train)
    shap_values = explainer(X_train)

    ##Show initial plot without combining one-hot-encoded
    if show_initial == True:
        shap.plots.beeswarm(shap_values, max_display=45, show=False)
        plt.title(f'{model_name} Raw SHAP Plot')
        plt.show()

    #### Combine JEWER ######
    shap_vals_jewer, sv_occ_jewer = combine_encoded(shap_values, 'Jewer-Boyd classification', 
                                              ['JEWER' in n for n in shap_values.feature_names])
    #### COMBINE SITE #######
    shap_vals_site, sv_occ_site = combine_encoded(shap_vals_jewer, 'Tumor site', 
                                             ['SITE' in n for n in shap_vals_jewer.feature_names])
    ###### COMBINE PLATE #######
    shap_vals_plate, sv_occ_plate = combine_encoded(shap_vals_site, 'Type of plate', 
                                              ['PLATE' in n for n in shap_vals_site.feature_names])
    #####COMBINE TXEXPOSURE #####
    shap_vals_tx, sv_occ_tx = combine_encoded(shap_vals_plate, 'Non-medication for plate exposure', 
                                              ['TXEXPOSURE' in n for n in shap_vals_plate.feature_names])
    ######SHOW SUB_PLOTS########
    if show_sub_plots == True:
        ##JEWER
        shap.plots.beeswarm(sv_occ_jewer, max_display=20, show=False)
        plt.title(f'{model_name} Jewer SHAP Plot')
        plt.show()
        ##SITE
        shap.plots.beeswarm(sv_occ_site, max_display=20, show=False)
        plt.title(f'{model_name} Site SHAP Plot')
        plt.show()
        ##PLATE
        shap.plots.beeswarm(sv_occ_plate, max_display=20, show=False)
        plt.title(f'{model_name} Plate SHAP Plot')
        plt.show()
        ##TXEXPOSURE
        shap.plots.beeswarm(sv_occ_tx, max_display=20, show=False)
        plt.title(f'{model_name} TXEXPOSURE SHAP Plot')
        plt.show()
    

    ############Get shap values, mean shap, sum shap########
    shap_array = shap_vals_tx.values
    feature_names = shap_vals_tx.feature_names
    shap_df = pd.DataFrame(shap_array, columns=feature_names)
    #Absolute AVG shap vals for each feature
    absolute_mean_shap = shap_df.abs().mean().reset_index()
    absolute_mean_shap.columns = ['Feature', 'Mean Absolute SHAP Value']
    if display_df == True:
        display(absolute_mean_shap)
    #Plot beeswarm plot
    shap.plots.beeswarm(shap_vals_tx, max_display=35, show=False)
    plt.title(f'{model_name}',  fontweight='semibold', fontsize = 25)
    plt.xlim(-0.7, 0.7)
    plt.show()

Use training data to get SHAP Plot

In [ ]:
get_shap(svc_clf, 'SVM', full_X_sm)

In [ ]:
get_shap(rf_clf, 'Random Forrest', full_X_sm)

In [ ]:
get_shap(tree_clf, 'Decision Tree', full_X_sm)

In [ ]:
get_shap(boost_clf, 'XGBoost', full_X_sm)

In [ ]:
get_shap(knn_clf, 'KNN', full_X_sm)

In [ ]:
get_shap(vote_clf, 'Vote', full_X_sm)

In [ ]:
get_shap(model_NN, 'Neural Network', nn_X_sm)

In [ ]:
get_shap(log_model, 'Nomogram', nomo_train_transformed)

# P-vals for AUROCS

**NOTE**: DeLong's test in R (using RStudio) from the pROC package was used to obtain p-values. Use the following code to get model predictions. Export these predictions and use the R code to get p-vals.

In [ ]:
print("----------------TRUE y-----------------")
display(raw_y_test.to_list())
##predict_proba() returns 2-d array, second entry is predicted prob of ORN occurance
y_proba_svm = svc_clf.predict_proba(ml_test_transformed)[:,1]
print('-----------------SVM----------------')
display(y_proba_svm)

y_proba_rf = rf_clf.predict_proba(ml_test_transformed)[:,1]
print('----------------RF----------------')
display(y_proba_rf)

y_proba_tree = tree_clf.predict_proba(ml_test_transformed)[:,1]
print('----------------TREE----------------')
display(y_proba_tree)

y_proba_boost = boost_clf.predict_proba(ml_test_transformed)[:,1]
print('----------------BOOST----------------')
display(y_proba_boost)

y_proba_knn = knn_clf.predict_proba(ml_test_transformed)[:,1]
print('----------------KNN----------------')
display(y_proba_knn)

y_proba_vote = vote_clf.predict_proba(ml_test_transformed)[:,1]
print('----------------VOTE----------------')
display(y_proba_vote)
#NN predict method only returns 1D array for prob of ORN
y_proba_nn = model_NN.predict(ml_test_transformed).ravel() 
print('----------------NN----------------')
display(y_proba_nn)
y_proba_nomo = log_model.predict_proba(nomo_test_transformed)[:,1]
print('----------------NOMO----------------')
display(y_proba_nomo)